<a href="https://colab.research.google.com/github/ElpoderdelComputo/SEI-Computo_aplicado/blob/main/MLP_2021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


>> # COLEGIO DE POSTGRADUADOS

INSTITUCIÓN DE ENSEÑANZA E INVESTIGACIÓN EN CIENCIAS AGRÍCOLAS

>>>> CAMPUS MONTECILLO

POSGRADO EN SOCIOECONOMÍA, ESTADÍSTICA E INFORMÁTICA

>>>> **CÓMPUTO APLICADO**


# **Clasificador: Perceptrón multicapa  E2021**

>**Autor:        José Rodríguez Rosales**

In [1]:
# -*- coding: utf-8 -*-

# Importar las librerias a utilizar
import pandas as pd
# Abre archivo >>> Train_RF_2021.csv  cargando archivo desde disco
#df= pd.read_csv("/content/Train_RF_2021.csv", header=0)
# Abre archivo >>> Train_RF_2021.csv desde google drive
df= pd.read_csv("/content/drive/MyDrive/Datos_2002_2021/Train_2021.csv", header=0)

#Mostrar primeros 5 registros
df[:5]

,X,Y,R,G,B,ExG,ExR,ExGR,CIVE,VEG,NGRDI,NDI,CLASE
0,26,89,62,76,76,14,10.8,3.2,28.95345,1.145453,0.101449,140.985507,4
1,24,88,66,80,81,13,12.4,0.6,30.39845,1.132214,0.095890,140.273973,4
2,26,87,67,81,81,14,12.8,1.2,30.02845,1.134926,0.094595,140.108108,4
3,27,87,67,82,79,18,11.8,6.2,28.04745,1.158543,0.100671,140.885906,4
4,80,61,70,71,73,-1,27.0,-28.0,34.78145,1.000211,0.007092,128.907801,4


In [ ]:
import numpy as np
# Valores de X: R,G,B,ExR,NGRDI
X = df.iloc[: , [2,3,4,5,10]].values
# y es la columna 12
y = df.iloc[: , 12].values

#imprime etiquetas de clases
print('Etiquetas de clase:', np.unique(y))
print(X[:5])

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

## División de datos en 90% de entrenamiento y 10% de datos de prueba:
# random_state = 1
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.10,
                                                    stratify=y,
                                                    random_state=1)

# Mostrar cuantos datos hay en total: en entrenamiento y prueba
print('\nNumero de etiquetas en y:', np.bincount(y))
print('Numero de etiquetas en y_train:', np.bincount(y_train))
print('Numero de etiquetas en y_test:', np.bincount(y_test))

# Estandarizar los datos
stdsc = StandardScaler()

X_train_std = stdsc.fit_transform(X_train)
X_test_std = stdsc.transform(X_test)

In [ ]:
# Búsqueda de hiperparámetros especificados para un estimador
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPClassifier

# Rango de valores de alpha
alpha = [0.001, 0.01, 0.1]

# Activacion
activation = ['relu', 'tanh']

# Neuronas en la capa oculta
hidden_layer_sizes = [50,100,150]

#
solver = ['lbfgs', 'sgd', 'adam']

#Modelo a optimizar
mlp = MLPClassifier(max_iter=3000,random_state=1)

# Crear retícula
param_grid = [{'alpha': alpha,
               'activation': activation,
               'hidden_layer_sizes': hidden_layer_sizes,
               'solver':solver}]

gs = GridSearchCV(estimator=mlp, 
                  param_grid=param_grid, 
                  scoring='accuracy', 
                  refit=True,
                  cv=10,
                  n_jobs=-1)

# Sea justan los modelos
gs = gs.fit(X_train_std, y_train)
# Mejor score
print(gs.best_score_)
# Mejores parametros
print(gs.best_params_)
# se obtiene mejor estimador 
clf = gs.best_estimator_

In [8]:
# Evaluamos el mejor estumador con X_test_std y y_test
print('Test accuracy: %.5f' % clf.score(X_test_std, y_test))

Test accuracy: 0.28867


In [ ]:
# Validacion cruzada con k=10 del mejor modelo ajustado 

from sklearn.metrics import precision_score, recall_score, f1_score,accuracy_score
from sklearn.metrics import make_scorer
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import cross_validate

# Metricas de evaluación con todos los datos 
scoring = {'prec_weighted': make_scorer(precision_score, average='macro'),
          'rec_weighted': make_scorer(recall_score, average='macro'),
          'f1_weighted': make_scorer(f1_score, average='macro'),
          'acc_weighted': make_scorer(accuracy_score)}


# Se utiliza mismo conjunto de datos
# División 90% de datos para entrenamiento, 10% de daatos para test
# random_state = 1 , k = 10
cvA = StratifiedShuffleSplit(n_splits=10, random_state=1)

# Se asignan las matrices de los datos de entrenamiento y prueba
crossVA = cvA.split(X_train_std, y_train)

# se inicia validacion
scores = cross_validate(estimator=clf,
                        X=X_train_std,y=y_train, 
                        scoring = scoring, 
                        return_estimator= True, 
                        cv = crossVA,
                        n_jobs=-1)

# Se imprimen los valores promedio y desviacion estandard de cada score
#print('CV scores: %s' % scores)
print('Accuracy: %.3f +/- %.3f' % (np.mean(scores['test_acc_weighted']),np.std(scores['test_acc_weighted'])))
print('F1 score: %.3f +/- %.3f' % (np.mean(scores['test_f1_weighted']),np.std(scores['test_f1_weighted'])))
print('Presicion: %.3f +/- %.3f' % (np.mean(scores['test_prec_weighted']),np.std(scores['test_prec_weighted'])))
print('Recall: %.3f +/- %.3f' % (np.mean(scores['test_rec_weighted']),np.std(scores['test_rec_weighted'])))
# se muestran los valores de cada score por cada particion
print("\n\t\tAccuracy\n")
print(scores['test_acc_weighted']) 
print("\n\t\t F1 \n")
print(scores['test_f1_weighted'])
print("\n\t\t Prec \n")
print(scores['test_prec_weighted'])
print("\n\t\t Rec \n")
print(scores['test_rec_weighted'])

In [ ]:
# Guardando el mejor modelo
import joblib
joblib.dump(clf, "./SVM_SelCa_2021.pkl")

['./RF_SelCa_2021.pkl']

In [11]:
import joblib
#importamos el mejor modelo ajustado 
clf = joblib.load('/content/MLP_SelCa_2021.pkl')

In [ ]:
# Mostrar Metricas por cada particion
ACC = scores['test_acc_weighted']
F1 = scores['test_f1_weighted']
PREC = scores['test_prec_weighted']
REC = scores['test_rec_weighted']

print('CV, Part, acc,  F1,   prec,   Rec')
for i in range(10):
  print('CV:',i+1  , '%.5f' % ACC[i], '%.5f' % F1[i],'%.5f' %  PREC[i],'%.5f' % REC[i])

In [ ]:
import matplotlib.pyplot as plt
import matplotlib
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import learning_curve

# Mostrar curvas de aprendizaje
# validacion cruzada k = 10
# Creamos tuberia COn el mejor modelo

train_sizes, train_scores, test_scores = learning_curve(estimator=clf,
                               X=X_train_std,
                               y=y_train,
                               train_sizes=np.linspace(0.1, 1.0, 10),
                               cv=10,
                               n_jobs=-1)

# Calculos de media y desviacion estandard
train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)
test_mean = np.mean(test_scores, axis=1)
test_std = np.std(test_scores, axis=1)

# Trazo de acc de entrenamiento
plt.plot(train_sizes, train_mean,
         color='blue', marker='o',
         markersize=5, label='ACC de entrenamiento')

# Trazo de media y desviacion estandard de etrenamiento
plt.fill_between(train_sizes,
                 train_mean + train_std,
                 train_mean - train_std,
                 alpha=0.15, color='blue')

# Trazo de acc de validación en verde
plt.plot(train_sizes, test_mean,
         color='green', linestyle='--',
         marker='s', markersize=5,
         label='ACC de la validación')

# Trazo de media y desviacion estandard de la validacion
plt.fill_between(train_sizes,
                 test_mean + test_std,
                 test_mean - test_std,
                 alpha=0.15, color='green')

plt.grid()
plt.title('RF2021')
plt.xlabel('Numero de Muestras de entrenamuento')
plt.ylabel('Precisión global')
plt.legend(loc='lower right')
plt.ylim([0.85, 1.0])
plt.tight_layout()
# plt.savefig('images/06_05.png', dpi=300)
plt.show()

In [ ]:
# Matriz de confusion normalizada
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

# Predecir las etiquetas de X_test_std
y_pred = clf.predict(X_test_std)
cm = confusion_matrix(y_test, y_pred, normalize='true')

# Prepara la matriz y agrega etiquetas
cm_display = ConfusionMatrixDisplay(cm, display_labels=['A','P','V','U'])
cm_display.plot()
# Agrega titulos y ejes y mostrar
cm_display.ax_.set(title='RF2021', xlabel='Clases predichas', ylabel='Clases verdaderas')

In [ ]:
# Mostrar metricas por clase
from sklearn.metrics import precision_recall_fscore_support as score

# Calcular los valores de la metricas
precision, recall, fscore, support = score(y_test, y_pred)

# Mostrar metricas
print('Precision: {}'.format(precision))
print('Recall: {}'.format(recall))
print('F1score: {}'.format(fscore))
# Valores calculados on numero de muestras
#print('Support: {}'.format(support))

In [ ]:
# Curvas ROC AUC
# ...................................................

# Bibliotecas necesarias
from itertools import cycle
from sklearn import svm, datasets
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from scipy import interp

n_classes = 4

# Predecir cada clase contra las otras
y_score = clf.fit(X_train_std, y_train).decision_function(X_test_std)

# Calcule la ROC de cada categoría
fpr = dict()
tpr = dict()
roc_auc = dict()

# Calcule la curva ROC macro-promedio y el área ROC 
y_test_dummies = pd.get_dummies(y_test, drop_first=False).values
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test_dummies[:, i], y_score[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Agregue todas las tasas de falsos positivos
all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))

# Interpolar todas las curvas ROC en estos puntos
mean_tpr = np.zeros_like(all_fpr)
for i in range(n_classes):
    mean_tpr += interp(all_fpr, fpr[i], tpr[i])

# Promediar y calcular el AUC macro
mean_tpr /= n_classes
fpr["macro"] = all_fpr
tpr["macro"] = mean_tpr
roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

# Trazar todas las curvas
lw=2
plt.figure()

plt.plot(fpr["macro"], tpr["macro"],
         label='Macro-promedio (AUC = {0:0.2f})'
               ''.format(roc_auc["macro"]),
         color='navy', linestyle=':', linewidth=4)

# Paleta de colores
colors = cycle(['red', 'green', 'blue', 'cyan'])

# Etiqueta de clases
clases = ['A','P','V','U']

# Trazo de las curvas por clase y agregar etiqueta de area AUC
for i, color in zip(range(n_classes), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=lw,
             label='Clase {0} (AUC = {1:0.2f})'
             ''.format(clases[i], roc_auc[i]))

# Trazo de la diagonal punteada
plt.plot([0, 1], [0, 1], 'k--', lw=lw)
plt.xlim([-0.05, 1.0])
plt.ylim([-0.05, 1.05])

# Etiquetas de los ejes y titulos
plt.xlabel('T F P')
plt.ylabel('T V P')
plt.title('Curvas ROC por clase (RF2021)')
plt.legend(loc="lower right")

# Tazar
plt.show()

In [ ]:
# Curvas P-S AUC 
# ...................................................
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
from sklearn.metrics import PrecisionRecallDisplay

# Diccionarios para cada clase: P, S y promedio
precision = dict()
recall = dict()
average_precision = dict()

# Calcule los valores de P  y S
for i in range(n_classes):
    precision[i], recall[i], _ = precision_recall_curve(y_test_dummies[:, i], y_score[:, i])
    average_precision[i] = average_precision_score(y_test_dummies[:, i], y_score[:, i])

# Calcular micro-promedio: cuantificación de la puntuación en todas las clases conjuntamente
precision["micro"], recall["micro"], _ = precision_recall_curve( y_test_dummies.ravel(), y_score.ravel())
average_precision["micro"] = average_precision_score(y_test_dummies, y_score, average="micro")

# Paleta de colores
colors = cycle(['red', 'green', 'blue', 'cyan', "teal"])

_, ax = plt.subplots(figsize=(6, 4))

# Crear la figura
display = PrecisionRecallDisplay(
    recall=recall["micro"],
    precision=precision["micro"],
    average_precision=average_precision["micro"],
)
display.plot(ax=ax, name="Micro-Promedio (AUC = ", color="K")

# Trazar cada curva por clase y etiquetas
for i, color in zip(range(n_classes), colors):
    display = PrecisionRecallDisplay(
        recall=recall[i],
        precision=precision[i],
        average_precision=average_precision[i],
    )
    display.plot(ax=ax, name=f"Clase {clases[i]} (AUC =", color=color)

# Configurar ejes
handles, labels = display.ax_.get_legend_handles_labels()
ax.set_xlim([0.0, 1.0])
ax.set_ylim([0.0, 1.05])

# Ubicar leyenda y titulo de la figura
ax.legend(handles=handles, labels=labels, loc="best")
ax.set_title("Curva P - S")
plt.xlabel('Sensibilidad')
plt.ylabel('Precisión')
# Trazar
plt.show()

In [ ]:
# Leer archivo con rgb e indices para predecir 
# X: R,G,B,ExR,NGRDI
#df= pd.read_csv("Huehuetla_todo_2021.csv",header=None) #base original
df= pd.read_csv("/content/drive/MyDrive/Datos_2002_2021/Huehuetla_todo_2021.csv",header=None) #base original

df.head()

In [ ]:
#Verificar tamaño de datos
X = df.iloc[: , [0,1,2,4,8]].values
np.size(X)/5

In [ ]:
# Predecir con el modelo ajustado
print('Leyedo archivo')

# Estandarizar datos
X_PRED_STD = stdsc.transform(X)

# Mostrar primeros registros de datos
print(X_PRED_STD[0:5])

# Predecir
y_pred = clf.predict(X_PRED_STD)

# Mostrar primeros registros de prediccion
print('\nListo!')

print(y_pred[0:10])

# Mostrar numero de clases
print('\nClases: 1 2 3 4 5')
 
# Cuantificar numero de muestras predichas por clase
print('CLasificacion:', np.bincount(y_pred))

print('\nFIN DE LA PREDICCION')

In [ ]:
# Agregamos a RGB el valor de su clase segun lo predicho
j=0
nuevo=[]
for i in X:
  nuevo.append([i[0],i[1],i[2], y_pred[j]])
  j+=1

# Reemplazamos la posicion 3 como 0 si el valor de R G Y B ES CERO
for i in nuevo:
  if i[0]==i[1]==i[2]==0:
    i[3]=0

#Agregamos un color a cada clase sustituyendo los valores RGB Segun sea el caso
for i in nuevo:
  if i[3]==1: # MAGENTA
    i[0]=255
    i[1]=0
    i[2]=255
  elif i[3]==2: # AMARILLO
    i[0]=255
    i[1]=255
    i[2]=0
  elif i[3]==3: # VERDE
    i[0]=6
    i[1]=137
    i[2]=2
  elif i[3]==4: # ROJO
    i[0]=255
    i[1]=0
    i[2]=0
  elif i[3]==0: # BLANCO 
    i[0]=255
    i[1]=255
    i[2]=255

# Eliminamos la cuarta columna para teer solo los 3 RGB
lista=[]
for i in nuevo:
  lista.append([i[0], i[1], i[2]])

In [ ]:
#Convertir lista a un arreglo 
arr = np.array(lista)
# Tamaño de la imagen original
ban = 3 # numero de bandas
m = 741 # ancho
n = 573 # alto
tamanio = m*n*ban #

# Mostrar tamaño de la lista y de la imagen deseada
print(tamanio)
print(m,n,ban)

# Convertir matriz de 1 dimension a 3
matriz = arr.reshape(n,m,3)
print(matriz.shape)

In [ ]:
# Crear imagen a partir de una matriz

from PIL import Image
from matplotlib import cm
import matplotlib.pyplot as plt

# Conveetir tipo de datos de una matriz a 'uint8'
img = np.array(matriz).astype('uint8')

# Crea imagen desde un arreglo
Huehue_img = Image.fromarray(img)

# Mostrar la imagen
Huehue_img.show()

# Agregarle etiquetas y mostrar la imagen
plt.imshow(Huehue_img)
plt.title(' Clasificación Huehuetla 2021 ') 
plt.axis('off')
plt.show()

In [ ]:
# Guardar la imagen generada
plt.imsave("CLF2021_SelCa.jpg",img)

In [ ]:
# Cuantificar numero de pixeles por clase
q,w,e,r,t = 0,0,0,0,0
for i in nuevo:
  if i[3]== 0:
    q+=1
  elif i[3]== 1:
    w+=1
  elif i[3]== 2:
    e+=1
  elif i[3]== 3:
    r+=1
  elif i[3]== 4:
    t+=1

#  Area por pixel
res = 15*15 #m2
# Crear una lista
tabla = [q,w,e,r,t]

In [ ]:
# Mostrar areas por clase en hectareas Totales
areas = tabla[1:]
area = 0
c = 1

print('Areas por clase\n')
for i in areas:
  h = i*res/10000
  print('Clase ', c, '=','%.2f' % h, 'ha')
  area = area+i*res/10000
  c += 1

print('\nHectáreas = ', '%.2f' % area, 'ha')